In [23]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:

def describe(key):
    val_lst =  []
    import statistics
    for i in scripts:
        for _key, val in i.items():
            if _key == key:
                val_lst.append(val)

    
    total = sum(val_lst)
    avg = statistics.mean(val_lst)
    s = statistics.stdev(val_lst)
    #sort the list for median and quartiles
    val_lst.sort()
    q25 = statistics.median(val_lst[:len(val_lst)//2])
    med = statistics.median(val_lst)
    q75 = statistics.median(val_lst[len(val_lst)//2:])

    return (total, avg, s, q25, med, q75)

In [10]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [11]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [12]:
bnf_names = set(i['bnf_name'] for i in scripts)
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [13]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    for key, val in script.items():
        if key == 'bnf_name':
            groups[val].append(script)

In [14]:
summary = {} 
for key, val in groups.items():
    items_lst = []
    for i in val:
        for _key, _val in i.items():
            if _key == 'items':
                items_lst.append(_val)
        
    summary[key] = sum(items_lst)

In [15]:
# {k: v for k, v in sorted(summary.items(), key=lambda item: item[1], reverse = True)}
[sorted(summary.items(), key=lambda item: item[1], reverse = True)][0][0]

('Omeprazole_Cap E/C 20mg', 113826)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [16]:
max_item = [("Omeprazole_Cap E/C 20mg", 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [17]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [18]:
def group_by_field(data, fields):
#     print (len(tuple(fields)))
        
    groups = {}
    from itertools import groupby 
    from operator import itemgetter 
    
    key = itemgetter(*fields) if isinstance(fields, tuple) else itemgetter(fields)
    data = sorted(data, key = key)
    for key, value in groupby(data, 
                          key = key):
        val_lst = []
        
        for k in value:
            val_lst.append(k)
        groups[key] = val_lst
    return groups

In [19]:
# scripts[0]

In [20]:
groups = group_by_field(scripts, ('bnf_name','nic'))
# [sorted(groups.items(), reverse = True)][0][0]

In [21]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = ...

assert test_max_item == max_item

AssertionError: 

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [22]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = ...
    else:
        practice_postal[practice['code']] = ...

In [23]:
def group_by_field_2(data, fields):
#     print (len(tuple(fields)))
        
    groups = {}
    from itertools import groupby 
    from operator import itemgetter 
    
    key = itemgetter(*fields) if isinstance(fields, tuple) else itemgetter(fields)
    data = sorted(data, key = key)
    for key, value in groupby(data, 
                          key = key):
        val_lst = []
        
        for k in value:
            val_lst.append(k)
        val_lst.sort(key = itemgetter('post_code'))
        val_lst = val_lst[0]['post_code']
        groups[key] = val_lst
    return groups

In [24]:
practice_postal = group_by_field_2(practices, 'code')

In [25]:
practice_postal

{'A81001': 'TS18 1HU',
 'A81002': 'TS18 2AW',
 'A81003': 'TS25 1QU',
 'A81004': 'TS1 3BE',
 'A81005': 'TS14 7DJ',
 'A81006': 'TS18 2AT',
 'A81007': 'TS24 7PW',
 'A81008': 'TS6 6TD',
 'A81009': 'TS5 6HF',
 'A81011': 'TS24 7PW',
 'A81012': 'TS3 6AL',
 'A81013': 'TS12 2FF',
 'A81014': 'TS23 2LA',
 'A81015': 'TS10 1TZ',
 'A81016': 'TS1 3QY',
 'A81017': 'TS17 0EE',
 'A81018': 'TS10 4NW',
 'A81019': 'TS3 7RL',
 'A81020': 'TS4 3BU',
 'A81021': 'TS6 6TD',
 'A81022': 'TS12 2TG',
 'A81023': 'TS1 2NX',
 'A81025': 'TS18 1HU',
 'A81026': 'TS5 6HA',
 'A81027': 'TS15 9DD',
 'A81029': 'TS1 2NX',
 'A81030': 'TS1 3RY',
 'A81031': 'TS24 7PW',
 'A81032': 'TS14 7DJ',
 'A81033': 'TS3 6AL',
 'A81034': 'TS17 0EE',
 'A81035': 'TS1 3RX',
 'A81036': 'TS20 2UZ',
 'A81037': 'TS1 2NX',
 'A81038': 'TS3 6AL',
 'A81039': 'TS16 9EA',
 'A81040': 'TS23 2DG',
 'A81041': 'TS24 9DN',
 'A81042': 'TS6 9QG',
 'A81043': 'TS6 0HA',
 'A81044': 'TS25 1QU',
 'A81045': 'TS10 1SR',
 'A81046': 'TS18 1YE',
 'A81047': 'TS11 6BW',
 'A810

In [26]:
practice_postal['K82019']

'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [27]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [28]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [29]:
items_by_post = group_by_field(joined, 'post_code')

In [30]:
summary = []
for key, val in items_by_post.items():
    items_lst = []
    for i in val:
        for _key, _val in i.items():
            if _key == 'items':
                items_lst.append(_val)
    
    summary.append((key, sum(items_lst)))
    
    

In [31]:
summary[:3]

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103)]

In [32]:
len(set(i[0] for i in summary))

#? the length of the answer should be 118

118

In [33]:
# postal_totals = [('B11 4BW', 20673)] * 100
postal_totals = summary[:100]
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [34]:
scripts[0:1]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'}]

In [35]:
total_items_by_bnf_post = group_by_field(joined, ('post_code', 'bnf_name'))
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [36]:
total_items_by_post = group_by_field(joined, 'post_code')

In [37]:
# total_items = ...
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [81]:
def summary(dic):
    summary = []
    for key, val in dic.items():
        items_lst = []
        for i in val:
            for _key, _val in i.items():
                if _key == 'items':
                    items_lst.append(_val)

        summary.append([key, sum(items_lst)])
    return summary

    

In [39]:
# summary(total_items_by_bnf_post)

In [ ]:
(1,2) + (3,5)

In [ ]:
print (list(total_items_by_post.keys())[:10])
# print (list(total_items_by_post.values())[:10])

In [100]:
summary_final = []
for key, val in total_items_by_post.items():
    val_g = group_by_field(val, 'bnf_name')
    sum_ = list(summary(val_g))
    for i in range(len(sum_)):
        sum_[i] = list(sum_[i])
        
        sum_[i].insert(0,key)
#         i = tuple(i)
    summary_final.append(sum_)

In [77]:
# summary_final1 = []
# for key, val in total_items_by_post.items():
#     val_g = group_by_field(val, 'bnf_name')
#     sum_ = list(summary(val_g))
# #     sum_ = sum_.append(key)
#     summary_final1.append(sum_)
            
    

In [107]:
# summary_final1[0]

In [101]:
summary_final[0]

[['B11 4BW', '3m Health Care_Cavilon Durable Barrier C', 7],
 ['B11 4BW', '3m Health Care_Cavilon No Sting Barrier', 2],
 ['B11 4BW', 'Abidec_Dps', 63],
 ['B11 4BW', 'Acetic Acid_Ear Spy 2% 5ml', 9],
 ['B11 4BW', 'Aciclovir_Crm 5%', 13],
 ['B11 4BW', 'Aciclovir_Tab 200mg', 4],
 ['B11 4BW', 'Aciclovir_Tab 800mg', 9],
 ['B11 4BW', 'Aciclovir_Tab Disper 400mg', 2],
 ['B11 4BW', 'Adalat LA 30_Tab 30mg', 3],
 ['B11 4BW', 'Adalat LA 60_Tab 60mg', 11],
 ['B11 4BW', 'Adapalene/Benzoyl Per_Gel 0.1%/2.5%', 3],
 ['B11 4BW', 'Adcal-D3_Capl 750mg/200u', 48],
 ['B11 4BW', 'Adcal-D3_Dissolve Tab Eff 1.5g/400u(Lem)', 8],
 ['B11 4BW', 'Adcal-D3_Tab Chble (Lem)', 7],
 ['B11 4BW', 'Adcal-D3_Tab Chble (Tutti-Frutti)', 48],
 ['B11 4BW', 'Adcal_Tab Chble 1.5g', 3],
 ['B11 4BW', 'Aerochamber Plus', 34],
 ['B11 4BW', 'Aerochamber Plus + Adult/Child/Infant Ma', 33],
 ['B11 4BW', 'Alendronic Acid_Tab 70mg', 54],
 ['B11 4BW', 'Alfacalcidol_Cap 1mcg', 7],
 ['B11 4BW', 'Alfacalcidol_Cap 250ng', 7],
 ['B11 4BW', 'A

In [143]:
def keeplargest(lst):
    from operator import itemgetter 
    lst.sort(key = itemgetter(2), reverse = True)
    return lst

new_summary = []
for list1 in summary_final:
    list1 = keeplargest(list1)[0]
    new_summary.append(list1)
        
    

In [148]:
new_summary

[('B11 4BW', 'Paracet_Tab 500mg', 451),
 ('B18 7AL', 'Metformin HCl_Tab 500mg', 421),
 ('B21 9RY', 'Salbutamol_Inha 100mcg (200 D) CFF', 706),
 ('B23 6DJ', 'Aspirin Disper_Tab 75mg', 449),
 ('B70 7AW', 'Lansoprazole_Cap 30mg (E/C Gran)', 775),
 ('BB11 2DL', 'Salbutamol_Inha 100mcg (200 D) CFF', 824),
 ('BB2 1AX', 'Simvastatin_Tab 40mg', 604),
 ('BB3 1PY', 'Influenza_Vac Inact 0.5ml Pfs', 1609),
 ('BB4 5SL', 'Salbutamol_Inha 100mcg (200 D) CFF', 652),
 ('BB7 2JG', 'Simvastatin_Tab 40mg', 990),
 ('BB8 0JZ', 'Omeprazole_Cap E/C 20mg', 1225),
 ('BB9 7SR', 'Paracet_Tab 500mg', 846),
 ('BD3 8QH', 'Lansoprazole_Cap 30mg (E/C Gran)', 601),
 ('BH18 8EE', 'Aspirin Disper_Tab 75mg', 886),
 ('BH23 3AF', 'Aspirin Disper_Tab 75mg', 756),
 ('BL1 8TU', 'Paracet_Tab 500mg', 498),
 ('BL3 5HP', 'Paracet_Tab 500mg', 676),
 ('BL9 0NJ', 'Salbutamol_Inha 100mcg (200 D) CFF', 619),
 ('BL9 0SN', 'Simvastatin_Tab 40mg', 874),
 ('CB9 8HF', 'Salbutamol_Inha 100mcg (200 D) CFF', 1150),
 ('CH1 4DS', 'Simvastatin_Ta

In [147]:
for i in range(len(new_summary)):
    new_summary[i] = tuple(new_summary[i])

In [115]:
# keeplargest([['B11 4BW', '3m Health Care_Cavilon Durable Barrier C', 7],
#  ['B11 4BW', '3m Health Care_Cavilon No Sting Barrier', 2],
#  ['B11 4BW', 'Abidec_Dps', 63]])

[['B11 4BW', 'Abidec_Dps', 63],
 ['B11 4BW', '3m Health Care_Cavilon Durable Barrier C', 7],
 ['B11 4BW', '3m Health Care_Cavilon No Sting Barrier', 2]]

In [120]:
total = summary(items_by_post)

In [142]:
new_summary

[['B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 7.990872240589406e-11],
 ['B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 8.104858139182133e-11],
 ['B21 9RY', 'Metformin HCl_Tab 500mg', 4.190704550040258e-11],
 ['B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 3.899203120057669e-11],
 ['B70 7AW', 'Paracet_Tab 500mg', 1.9999536622309116e-11],
 ['BB11 2DL', 'Omeprazole_Cap E/C 20mg', 2.4438011918558703e-11],
 ['BB2 1AX', 'Omeprazole_Cap E/C 20mg', 4.5666465805809016e-11],
 ['BB3 1PY', 'Omeprazole_Cap E/C 20mg', 1.1536794369202698e-11],
 ['BB4 5SL', 'Omeprazole_Cap E/C 20mg', 4.7121714769244445e-11],
 ['BB7 2JG', 'Omeprazole_Cap E/C 20mg', 1.4826172620591956e-11],
 ['BB8 0JZ', 'Atorvastatin_Tab 20mg', 7.630042428897668e-12],
 ['BB9 7SR', 'Omeprazole_Cap E/C 20mg', 1.6312103927664864e-11],
 ['BD3 8QH', 'Atorvastatin_Tab 40mg', 7.752659913598205e-11],
 ['BH18 8EE', 'Omeprazole_Cap E/C 20mg', 1.866928778780649e-11],
 ['BH23 3AF', 'Omeprazole_Cap E/C 20mg', 3.524710483989678e-11],
 ['BL1 8T

In [121]:
total[0]

['B11 4BW', 20673]

In [137]:
def search_total(post_code):
    for i in total:
        if i[0] == post_code:
            val = i[1]
    return val
        
new_summary_portion = []
for i in new_summary:
    i[2] = float(i[2]/search_total(i[0]))
    new_summary_portion.append(i)
    

In [138]:
for i in range(len(new_summary_portion)):
    new_summary_portion[i] = tuple(new_summary_portion[i])

In [139]:
new_summary_portion

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 7.990872240589406e-11),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 8.104858139182133e-11),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 4.190704550040258e-11),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 3.899203120057669e-11),
 ('B70 7AW', 'Paracet_Tab 500mg', 1.9999536622309116e-11),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 2.4438011918558703e-11),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 4.5666465805809016e-11),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 1.1536794369202698e-11),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 4.7121714769244445e-11),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 1.4826172620591956e-11),
 ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 7.630042428897668e-12),
 ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 1.6312103927664864e-11),
 ('BD3 8QH', 'Atorvastatin_Tab 40mg', 7.752659913598205e-11),
 ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 1.866928778780649e-11),
 ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 3.524710483989678e-11),
 ('BL1 8T

In [118]:
len(new_summary)  

118

In [ ]:
max_item_by_post = ...

In [151]:
# items_by_region = new_summary_portion[:100]
items_by_region = new_summary[:100]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [8]:
def group_by_field(data, fields):
    names = {tuple(item[field] for field in fields) for item in data}
    groups = {name: [] for name in names}
    for item in data:
        name = tuple(item[field] for field in fields)
        groups[name].append(item)
    return groups

In [10]:
practice_postal = {}
for practice in practices:
    code = practice['code']
    if code not in practice_postal:
        practice_postal[code] = practice['post_code']
    else:
        practice_postal[code] = min(practice_postal[code], practice['post_code'])

In [11]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

pc_grouped = list(group_by_field(joined, ('post_code',)).items())

In [12]:
name_grouped = group_by_field(joined, ('post_code','bnf_name')).items()
total_items_by_bnf_post= {name: sum(group['items'] for group in groups) for name, groups in name_grouped}
assert len(total_items_by_bnf_post) == 141196

In [13]:
total_items= [{'post_code': tup[0], 'bnf_name': tup[1], 'total' : tot} for tup, tot in list(total_items_by_bnf_post.items())]

total_items_by_post = group_by_field(total_items, ('post_code',))


assert len(total_items_by_post) == 118

In [16]:
items_by_post = [(pc[0], sum(group['items'] for group in groups)) for pc, groups in pc_grouped]
items_by_post=sorted(items_by_post)

In [17]:
max_items_by_post = [sorted(group, key=lambda mem: (-mem['total'], mem['bnf_name']))[0] for group in total_items_by_post.values()]


In [18]:
result= [(item['post_code'], item['bnf_name'],item['total'] /dict(items_by_post)[item['post_code']]) for item in max_items_by_post]

In [19]:
result= [(item['post_code'], item['bnf_name'],item['total'] /dict(items_by_post)[item['post_code']]) for item in max_items_by_post]

# Take the first 100 elements and pass it to the grader
items_by_region = sorted(result)[:100]

In [104]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [20]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*